In [96]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Volumes/UsersHome/linglv/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [87]:
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Georgia%20Tech%2C%20Atlanta%2C%20GA&ns=1&l=g%3A-84.41276550292969%2C33.76816006007425%2C-84.38718795776367%2C33.78956253387451'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [134]:
import time
result = []
while True:
    eles = soup.find_all(
        "div", class_=["container__09f24__sxa9-", "hoverable__09f24__3HkY2"]
    )
    for ele in eles:
        name = ele.find_all("div", class_="businessName__09f24__3Ml2X")[0].text
        reviewer = None
        price = None
        restauran_type = None
        star = None

        try:
            reviewer = ele.find_all(
                "span", class_="reviewCount__09f24__3GsGY css-e81eai"
            )[0].text
        except:
            pass

        try:
            price = ele.find_all("span", class_="priceRange__09f24__2GspP css-e81eai")[
                0
            ].text
        except:
            pass

        try:
            types = ele.find_all("p", class_=["text__09f24__2NHRu", "css-1hx6l2b"])
            l = len(types)
            for i in l:
                restauran_type.append(types[i])
        except:
            pass

        try:
            star = ele.find_all("div", class_="i-stars__09f24___sZu0")[0]["aria-label"]
        except:
            pass
        result.append({
            'name': name,
            'reviewer': reviewer,
            'price': price,
            'restauran_type': restauran_type,
            'star': star
        })
    try:
        time.sleep(1)
        browser.find_by_css(".next-link.navigation-button__09f24__1EzzD.css-166la90").click()
    except:
        print('end of the list')
        break


end of the list


<IPython.core.display.Javascript object>